In [1]:
%matplotlib inline
import importlib
from pytorch_utils_oh_2 import *

Pytorch utils oh: pytorch_utils_oh_2.py
Pytorch: 0.2.0_4


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
MODEL_SAVE_PATH = 'whole_gen_4_chars_wordvect'

In [4]:
import pytorch_utils_oh_2; importlib.reload(pytorch_utils_oh_2); from pytorch_utils_oh_2 import *;

Pytorch utils oh: pytorch_utils_oh_2.py
Pytorch: 0.2.0_4


# Data loading

In [5]:
all_data = pickle.load(open("data/en_train_fixed_1.pkl", "rb" ))
all_data_sentence_index = all_data.set_index('sentence_id')

In [6]:
categories_all = sorted(all_data["class"].unique())
print(categories_all)
print(len(categories_all))
categories_index = dict((c, i) for i, c in enumerate(categories_all))

['ELECTRONIC', 'LETTERS', 'NOT_CHANGED', 'NUMBERS', 'PLAIN', 'VERBATIM']
6


### Utils stuff

In [7]:
chars_normal, chars_normal_index = load_characters_pkl('data/en_features/chars_normal.pkl')
print(''.join(chars_normal))

<SOS><EOS>☒ !"#$%&'(),-./0123456789:;ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyz~£¥ª²³µº¼½¾éɒʻˈΩμ—€⅓⅔⅛


common_words, common_words_index = load_common_words_10k()
len(common_words)
common_words[0:10]

In [8]:
wv_vecs, wv_words, wv_idx = load_glove('/home/ohu/koodi/data/glove_wordvec/glove.6B.50d.txt')

### More balanced sample

In [9]:
sample_data = all_data[all_data['class'] != 'NOT_CHANGED']
print("Data rows: {},  (dropped rows: {})".format(len(sample_data), len(all_data)-len(sample_data)))
sample_data = sample_data.reset_index(drop=True)

Data rows: 659544,  (dropped rows: 9258648)


In [10]:
balanced_data_classes_select = list(sample_data.groupby('class'))

balanced_data_accessed_counter = 0 
balanced_data_length = 0
def balanced_data_randomize(max_len=20000):
    global balanced_data, balanced_data_length, balanced_data_accessed_counter
    balanced_data = pd.concat([v.sample(min(max_len, len(v))) for k, v in balanced_data_classes_select])
    balanced_data_length = len(balanced_data)
    balanced_data_accessed_counter = 0

def balanced_data_sample_row():
    global balanced_data_accessed_counter
    global balanced_data_last_sample
    balanced_data_accessed_counter += 1
    if balanced_data_accessed_counter/balanced_data_length > 0.2:
        balanced_data_randomize()
    balanced_data_last_sample = balanced_data.iloc[random.randint(1, balanced_data_length-1)]
    return balanced_data_last_sample
    
balanced_data_randomize()

In [11]:
%%timeit
balanced_data_sample_row()

124 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [12]:
#all_data.groupby('class')['class'].count()
#sample_data.groupby('class')['class'].count()
balanced_data.groupby('class')['class'].count()

class
ELECTRONIC     4964
LETTERS       20000
NUMBERS       20000
PLAIN         20000
VERBATIM      16950
Name: class, dtype: int64

In [13]:
 balanced_data_sample_row()

sentence_id    25106
token_id           7
class          PLAIN
before            st
after          saint
class_org      PLAIN
Name: 21947, dtype: object

### After words

In [14]:
tmp = list(set(sample_data['after']))
chars_after = sorted(list(set(''.join(tmp))))
chars_after = [SOS_TOKEN, EOS_TOKEN, UNKNOWN_CHAR] + chars_after
chars_after_index = dict((c, i) for i, c in enumerate(chars_after))
len(chars_after)
'_'.join(chars_after)
chars_after_index['z']

32

"<SOS>_<EOS>_☒_ _'_a_b_c_d_e_f_g_h_i_j_k_l_m_n_o_p_q_r_s_t_u_v_w_x_y_z_é"

30

In [15]:
pickle.dump(chars_after, open('data/en_features/chars_after_1.pkl', 'wb'))

In [16]:
chars_after = pickle.load(open('data/en_features/chars_after_1.pkl', 'rb'))
chars_after_index = dict((c, i) for i, c in enumerate(chars_after))

In [17]:
def after_string_to_tensor(word, include_eos=True):
    return string_to_tensor(word, chars_index=chars_after_index, include_eos=include_eos)
after_string_to_tensor('abcé').shape
after_string_to_tensor('abcé')[0, -2, -1] == 1

torch.Size([1, 5, 32])

True

In [18]:
onehot_sos = after_string_to_tensor([SOS_TOKEN], include_eos=False)
onehot_sos.size()

torch.Size([1, 1, 32])

### Samples

In [19]:
def get_random_sample():
    sample_row = balanced_data_sample_row()
    sentence_id = sample_row['class']

    rows = all_data_sentence_index.loc[sample_row['sentence_id']]
    befores = list(rows.before)
        
    token_id_idx = list(rows['token_id']).index(sample_row['token_id'])
    befores[token_id_idx] = SAMPLE_WORD_TOKEN
    
    return sample_row['before'], sample_row['after'], sample_row['class'], befores
            
def tmp():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    print(s_class, ':', s_bef, '->', s_aft)
    print(' '.join(s_sentence))
    print(s_sentence)
    print(string_to_tensor(s_bef, chars_normal_index).shape)
    print(after_string_to_tensor(s_aft.split(' ')).shape)
tmp()

NUMBERS : 7 -> seven
In Ireland in 2007 , <SAMPLE> Up launched a range of flavored water .
['In', 'Ireland', 'in', '2007', ',', '<SAMPLE>', 'Up', 'launched', 'a', 'range', 'of', 'flavored', 'water', '.']
torch.Size([1, 2, 104])
torch.Size([1, 2, 32])


# Model functions

In [20]:
use_cuda = True

### Encoder

In [21]:
class EncoderRNN(nn.Module):
    def __init__(self, words_input_size, chars_input_size, words_hidden_size, chars_hidden_size,
                 words_layers=1, chars_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.words_layers = words_layers
        self.chars_layers = chars_layers
        self.words_hidden_size = words_hidden_size
        self.chars_hidden_size = chars_hidden_size

        self.rnn_words = nn.LSTM(words_input_size, words_hidden_size // 2, words_layers,
                                 batch_first=True, bidirectional=True)

        self.rnn_chars = nn.LSTM(chars_input_size, chars_hidden_size // 2, chars_layers,
                                batch_first=True, bidirectional=True)
        
    def forward(self, word_vectors, string_tensor, hidden = None, init_hidden = True):
        if init_hidden:
            hidden_words, hidden_chars = self.init_hidden()
        
        all_outputs_words, hidden_words = self.rnn_words(word_vectors, hidden_words)
        output_words = all_outputs_words[:, -1]
        
        all_outputs_chars, hidden_chars = self.rnn_chars(string_tensor, hidden_chars)
        output_chars = all_outputs_chars[:, -1]
        
        output = torch.cat((output_words, output_chars), 1)
        
        return output

    def init_hidden(self):
        var1_1 = Variable(torch.zeros(2 * self.words_layers, 1, self.words_hidden_size // 2))
        var1_2 = Variable(torch.zeros(2 * self.words_layers, 1, self.words_hidden_size // 2))
        var2_1 = Variable(torch.zeros(2 * self.chars_layers, 1, self.chars_hidden_size // 2))
        var2_2 = Variable(torch.zeros(2 * self.chars_layers, 1, self.chars_hidden_size // 2))
        
        var1_1 = var1_1.cuda(); var1_2 = var1_2.cuda()
        var2_1 = var2_1.cuda(); var2_2 = var2_2.cuda()
        return ((var1_1, var1_2), (var2_1, var2_2))

In [22]:
encoder_rnn = EncoderRNN(words_input_size=wv_vecs.shape[-1], chars_input_size=len(chars_normal),
                         words_hidden_size=128, chars_hidden_size=256,
                         words_layers=2, chars_layers=2).cuda()
encoder_rnn

EncoderRNN (
  (rnn_words): LSTM(50, 64, num_layers=2, batch_first=True, bidirectional=True)
  (rnn_chars): LSTM(104, 128, num_layers=2, batch_first=True, bidirectional=True)
)

In [23]:
def test_encoder_single_sample():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    
    words_t = Variable(words_to_word_vectors_tensor(list(s_sentence), wv_vecs, wv_idx)).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    return encoder_rnn(words_t, string_t)
    
encoder_output = test_encoder_single_sample()
encoder_output.size()

torch.Size([1, 384])

### Decoder

In [24]:
class DecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        
        self.rnn = nn.GRU(input_size, hidden_size, n_layers,
                                 batch_first=True, bidirectional=False)
        
        self.lin_out = nn.Linear(hidden_size, input_size)
        #self.softmax = nn.LogSoftmax()

    def forward(self, char, hidden):
        #char = char.view(1,1,-1)
        #hidden = hidden.view(1,1,-1)
        output, hidden = self.rnn(char, hidden)
        output = output[:, -1] # view(1,-1)
        output = self.lin_out(output)
        output = F.log_softmax(output)
        return output, hidden
    
    def init_rest_hidden(self, input_var):
        hid_var = Variable(torch.zeros(self.n_layers - 1, 1, self.hidden_size)).cuda()
        res = torch.cat((input_var, hid_var), 0)
        return res
        

decoder_rnn = DecoderRNN(input_size=len(chars_after), hidden_size=128+256, n_layers=2)
decoder_rnn = decoder_rnn.cuda()
decoder_rnn

DecoderRNN (
  (rnn): GRU(32, 384, num_layers=2, batch_first=True)
  (lin_out): Linear (384 -> 32)
)

In [25]:
tmp_hiddens = decoder_rnn.init_rest_hidden(encoder_output.view(1,1,-1))
tmp_a, tmp_b = decoder_rnn(Variable(onehot_sos).cuda(), tmp_hiddens)
print(tmp_a.size())
print(tmp_b.size())
print(tmp_a.topk(1)[1])
print(chars_after[tmp_a.topk(1)[1].data[0][0]])

torch.Size([1, 32])
torch.Size([2, 1, 384])
Variable containing:
 24
[torch.cuda.LongTensor of size 1x1 (GPU 0)]

t


# Training etc

### Accuracy

In [26]:
def test_model_single_sample(model=None):
    s_bef, s_aft, s_class, s_sentence = sample = get_random_sample()
        
    words_t = Variable(words_to_word_vectors_tensor(list(s_sentence), wv_vecs, wv_idx)).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    encoder_output = encoder_rnn(words_t, string_t)
    
    encoder_output = encoder_output.view(1,1,-1)
    
    decoder_hidden = decoder_rnn.init_rest_hidden(encoder_output)
    decoder_input = Variable(onehot_sos).cuda()

    decoded_output = []
    max_length = 20
    for _ in range(max_length):
        decoder_output, decoder_hidden = decoder_rnn(decoder_input, decoder_hidden)
        #return decoder_output

        topv, topi = decoder_output.data.topk(1)
        char_index = topi[0][0]
        char = chars_after[char_index] # Use own prediction as next input
                
        if char == EOS_TOKEN:
            break

        decoded_output.append(char)
        
        decoder_input = after_string_to_tensor([char], include_eos=False)
        decoder_input = Variable(decoder_input).cuda()
    
    output = ''.join(decoded_output)
    return output, output, s_aft, sample
    
tmp = test_model_single_sample(None)
tmp

('tttttttttttttttttttt',
 'tttttttttttttttttttt',
 'and',
 ('&',
  'and',
  'VERBATIM',
  ['Smith', ',', 'A.L.', '<SAMPLE>', 'Rea', ',', 'C.', '(', '1906', ')', '.']))

In [27]:
def print_local_wrong_predictions(max_results=10):
    arr = get_some_wrong_predictions(None, test_model_single_sample, max_iterations=10000, max_results=max_results)
    for sample, predict, output in arr:
        s_bef, s_aft, s_class, s_sentence = sample
        print("{:<14} => {:<14} || {} \n{:>17} {}".format(s_bef, predict, s_aft, '', ' '.join(s_sentence), ))


In [28]:
print_local_wrong_predictions(2)

J.             => tttttttttttttttttttt || j 
                  In the spring of 1926 , <SAMPLE> Grossman's Sons sold the Grapico business to New Orleans business Pan American Manufacturing Co .
50             => tttttttttttttttttttt || fifty 
                  " The <SAMPLE> Greatest Rock Frontmen of All Time " .


In [29]:
%%time
test_model_accuracy(encoder_rnn, test_model_single_sample, n_sample=1000)

Accuracy: 0.00% (       0/    1000)
CPU times: user 16.8 s, sys: 552 ms, total: 17.4 s
Wall time: 17.4 s


0.0

### Training

In [30]:
def train(s_bef, s_aft, s_sentence, encoder_optimizer, decoder_optimizer, loss_function,
          use_teacher_forcing, max_length=20):
    
    words_t = Variable(words_to_word_vectors_tensor(list(s_sentence), wv_vecs, wv_idx)).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    encoder_output = encoder_rnn(words_t, string_t)
    encoder_output = encoder_output.view(1,1,-1)
    
    decoder_hidden = decoder_rnn.init_rest_hidden(encoder_output)
    decoder_input = Variable(onehot_sos).cuda()
    
    ###
    
    target_arr = list(s_aft) + [EOS_TOKEN]
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0
    
    decoded_output = []
    for i in range(len(target_arr)):
        decoder_output, decoder_hidden = decoder_rnn(decoder_input, decoder_hidden)

        decoder_target_i = chars_after_index[target_arr[i]]
        decoder_target_i = Variable(torch.LongTensor([decoder_target_i])).cuda()
        loss += loss_function(decoder_output, decoder_target_i)
        
        topv, topi = decoder_output.data.topk(1)
        char_index = topi[0][0]
        char = chars_after[char_index] # Use own prediction as next input
        decoded_output.append(char)
        
        if use_teacher_forcing:
            char = target_arr[i] # replace input with right target
        else:
            # use output normally as input 
            if char == EOS_TOKEN:
                break
                
        decoder_input = after_string_to_tensor([char], include_eos=False)
        decoder_input = Variable(decoder_input).cuda()
        
    if decoded_output[-1] == EOS_TOKEN:
        decoded_output = decoded_output[:-1]
        
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return ''.join(decoded_output), (loss.data[0] / len(target_arr))


In [31]:
def train_iterations(n_iters=100000, lr=0.001, teacher_forcing_ratio=0.5,
                     print_every=10000, plot_every=1000):

    start = time.time()
    
    decoder_rnn.train()
    encoder_rnn.train()

    current_loss = 0
    current_loss_iter = 0

    encoder_optimizer = torch.optim.Adam(encoder_rnn.parameters(), lr=lr)
    decoder_optimizer = torch.optim.Adam(decoder_rnn.parameters(), lr=lr)
    loss_function = nn.NLLLoss()
    
    for iteration in range(1, n_iters + 1):
        model_training.iterations += 1
        
        use_teacher_forcing = random.random() < teacher_forcing_ratio
        
        s_bef, s_aft, s_class, s_sentence = get_random_sample()
        
        result, loss = train(s_bef=s_bef, s_aft=s_aft, s_sentence=s_sentence,
                             encoder_optimizer=encoder_optimizer, decoder_optimizer=decoder_optimizer,
                             loss_function=nn.NLLLoss(), use_teacher_forcing=use_teacher_forcing,
                             max_length=40 )
        
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            teacher_forcing_str = ""
            if use_teacher_forcing:
                teacher_forcing_str = "(forcing)"
            correct = '✓' if result == s_aft else "✗: {}".format(s_aft)
            
            print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {} -> {} ({}) {}".format(
                      model_training.iterations, iteration/n_iters, time_since(start),
                      current_loss/current_loss_iter, loss,
                      s_bef, result, correct, teacher_forcing_str))

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_training.losses.append(current_loss / plot_every)
            model_training.learning_rates.append(lr)
            current_loss = 0
            current_loss_iter = 0
            
        if model_training.iterations % 50000 == 0 or model_training.iterations == 10:
            model_training.save_models()
            acc = test_model_accuracy(encoder_rnn, test_model_single_sample)
            model_training.accuracy.append(acc)
    
    # test_model_accuracy(model, n_sample=10000)

In [32]:
model_training = ModelTraining(MODEL_SAVE_PATH, [encoder_rnn, decoder_rnn])

Save path: data/models/whole_gen_4_chars_wordvect


In [33]:
train_iterations(n_iters=50, print_every=9, lr=0.0001)

     9  18% (   0m 0s)   3.459   |   3.45: & -> tttt (✗: and) (forcing)
Saved model to data/models/whole_gen_4_chars_wordvect/10_(EncoderRNN/DecoderRNN)
Accuracy: 0.00% (       0/   10000)
    18  36% (  2m 53s)   3.454   |   3.43: 2 August 2007 -> ttttt                          tttt      (✗: the second of august two thousand seven) (forcing)
    27  54% (  2m 53s)   3.444   |   3.43: ς -> tttt   (✗: sigma) (forcing)
    36  72% (  2m 54s)   3.170   |   2.59: Mk -> tt (✗: m k) 
    45  90% (  2m 54s)   3.096   |   3.37: February 12, 1898 -> t                                       (✗: february twelfth eighteen ninety eight) 


In [34]:
train_iterations(n_iters=(1000-model_training.iterations), print_every=500, lr=0.0001)

   550  53% (  0m 17s)   2.903   |   3.27: Harbour ->         (✗: harbor) (forcing)


In [35]:
train_iterations(n_iters=9000, lr=0.0001, print_every=1000)

  2000  11% (  0m 35s)   2.437   |   2.10: 9 -> tod (✗: nine) 
  3000  22% (   1m 7s)   2.249   |   0.86: & -> and (✓) 
  4000  33% (  1m 39s)   2.023   |   0.99: 2000 -> t n t (✗: two thousand) 
  5000  44% (  2m 12s)   1.952   |   1.82: MVP -> s s t (✗: m v p) (forcing)
  6000  56% (  2m 45s)   1.842   |   2.86: ο -> tom<EOS>a<EOS><EOS> (✗: omicron) (forcing)
  7000  67% (  3m 18s)   1.727   |   2.15: R. -> t (✗: r) 
  8000  78% (  3m 53s)   1.704   |   1.25: Theatre -> teeeter (✗: theater) (forcing)
  9000  89% (  4m 25s)   1.646   |   3.08: December 14, 2008 -> tee teenty tinety tinety tinety tine (✗: december fourteenth two thousand eight) 
 10000 100% (  4m 56s)   1.564   |   1.57: A- -> s (✗: a) (forcing)


In [36]:
train_iterations(n_iters=90000, print_every=10000)

 20000  11% (  5m 31s)   1.229   |   2.46: 13 -> fifty fif (✗: thirteen) 
 30000  22% ( 10m 59s)   0.899   |   0.21: vol -> volume (✓) 
 40000  33% ( 16m 28s)   0.704   |   0.00: & -> and (✓) (forcing)
 50000  44% (  22m 0s)   0.623   |   0.00: - -> to (✓) 
Saved model to data/models/whole_gen_4_chars_wordvect/50000_(EncoderRNN/DecoderRNN)
Accuracy: 56.46% (    5646/   10000)
 60000  56% ( 29m 19s)   0.533   |   0.02: CB- -> c b (✓) (forcing)
 70000  67% (  35m 5s)   0.477   |   0.05: 0 -> zero (✓) (forcing)
 80000  78% ( 40m 44s)   0.469   |   0.09: 1998 -> nineteen ninety eight (✓) 
 90000  89% ( 46m 23s)   0.400   |   0.01: MK -> m k (✓) 
100000 100% ( 51m 55s)   0.445   |   0.01: st -> saint (✓) (forcing)
Saved model to data/models/whole_gen_4_chars_wordvect/100000_(EncoderRNN/DecoderRNN)
Accuracy: 67.71% (    6771/   10000)


In [37]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.001)

110000   3% (  5m 35s)   0.422   |   0.03: N.E. -> n e (✓) 
120000   7% ( 11m 11s)   0.387   |   0.02: 24 -> twenty four (✓) 
130000  10% ( 16m 52s)   0.364   |   0.00: jr -> junior (✓) 
140000  13% ( 22m 31s)   0.341   |   0.94: LewRockwell.com -> l e r e e l e e e l e eot c o m  (✗: l e w r o c k w e l l dot c o m) (forcing)
150000  17% (  28m 6s)   0.362   |   0.45: ab11e5b4-ccb1-430e-9a7c-598d63c7480b.gif -> april seventeenth o h o o eight (✗: a b e l e v e n e f i v e b f o u r d a s h c c b o n e d a s h f o u r t h r e e o e d a s h n i n e a s e v e n c d a s h f i v e n i n e e i g h t d s i x t y t h r e e c s e v e n f o u r e i g h t o b dot g i f) 
Saved model to data/models/whole_gen_4_chars_wordvect/150000_(EncoderRNN/DecoderRNN)
Accuracy: 70.79% (    7079/   10000)
160000  20% ( 35m 25s)   0.390   |   0.00: - -> to (✓) (forcing)
170000  23% (  41m 5s)   0.380   |   0.03: 1888 -> eighteen eighty eight (✓) (forcing)
180000  27% ( 46m 46s)   0.358   |   0.03: metres -> met

IndexError: list index out of range

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.3, lr=0.001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0.3, lr=0.0005)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=100000, print_every=10000, teacher_forcing_ratio=0.1, lr=0.0005)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.1, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()